## Zadania do wykonania

### Zadanie 1

Zaprojektuj klasę abstrakcyjną / rekord, która wyznacza pole, obwód oraz zwraca długości boków trójkąta (dowolnego). Klasami dziedziczącymi z tej klasy ma być trójkąt:
* równoboczny (konstruktor z jednym parametrem),
* równoramienny (konstruktor z dwoma parametrami - dwa różne boki),
* prostokątny (konstruktor z z dwoma parametrami - długości przyprostokątne).

> Przeciąż metodę `ToString()`, która ma zawierać zestaw podstawowych informacji. Zadanie można zrealizować na dwa sposoby: 
1. metody do obliczeń pola i obwodu zaimplementować w klasie abstrakcyjnej używając wzoru:

$
	P =\sqrt{p\cdot(p-A)\cdot(p-B)\cdot(p-C)}
$

gdzie: $p$ to połowa obwodu trójkąta. Symbole $A$, $B$, $C$ to długości poszczególnych boków. Oznaczyć metody obliczające pole i obwód jako abstrakcyjne i przeciążyć je w klasie szczegółowej.

Metoda `ToString` powinna być przeciążona w klasie abstrakcyjnej.

In [3]:
public abstract class Triangle
{
    protected readonly double a;
    protected readonly double b;
    protected readonly double c;
    protected readonly double p;

    protected Triangle(double a, double b, double c)
    {
        this.a = a;
        this.b = b;
        this.c = c;
        this.p = (a + b + c) / 2;
    }

    public double Area()
    {
        return Math.Sqrt(p * (p - a) * (p - b) * (p - c));
    }

    public double Perimeter()
    {
        return a + b + c;
    }

    public override string ToString()
    {
        return string.Format("a: {0}, b: {1}, c: {2}, area: {3}, perimeter: {4}", a, b, c, Area(), Perimeter());
    }
}

// trójkąt równoboczny
public class EquilateralTriangle : Triangle
{
    public EquilateralTriangle(double a) : base(a, a, a) { }
}

// trójkąt równoramienny
public class IsoscelesTriangle : Triangle
{
    public IsoscelesTriangle(double a, double b) : base(a, b, b) { }
}

// trójkąt prostokątny
public class RightTriangle : Triangle
{
    public RightTriangle(double a, double b) : base(a, b, Math.Sqrt(a * a + b * b)) { }
}



### Zadanie 2

Utwórz klasę generyczną kartoteka pracowników, która umożliwia:

* dodawanie/usuwanie,
* wyświetlanie,
* walidację istniejących już pracowników,
* wyszukiwanie danych. 

Klasa pracownik musi zawierać przynajmniej 5 cech. Każdy pracownik powinien posiadać stanowisko pracy. Klasa powinna zawierać walidację danych (poprzez metodę `Validate()`). Klasa pracownik powinna zawierać metody `Show()` i `IsMatch()`, z których będzie korzystała kartoteka przy wyszukiwaniu.

Przy wyświetlaniu pomocna może okazać się metoda `Format()`.


In [4]:
using System.Collections;
using System.Text.Json;
using System.Text.RegularExpressions;
using System.Xml.Serialization;
using System.IO;

class CatalogGeneric<T>
{
    private List<T> catalog = [];

    public void Add(T item) { catalog.Add(item); }
    public void AddMany(List<T> items) { catalog.AddRange(items); }
    public void Remove(T item) { catalog.Remove(item); }

    public void Print()
    {
        if (catalog.Count == 0)
        {
            Console.WriteLine("Catalog is empty");
        }
        else
        {
            foreach (T item in catalog)
            {
                Console.WriteLine(item?.ToString());
            }
        }
    }

    public T? Search(T catalogItem) { return (catalog.Count != 0 && catalog.Contains(catalogItem)) ? catalogItem : default(T); }
    public void Clear() { catalog.Clear(); }
    public bool Validate(T item) { return catalog.Contains(item); }

}


public class Employee
{

    public string Name { get; set; }
    public string Surname { get; set; }
    public string Position { get; set; }
    public string Email { get; set; }
    public int Id { get; set; }

    public Employee()
    {
    }

    public Employee(string name, string surname, string position, string email, int id)
    {
        Name = name;
        Surname = surname;
        Position = position;
        Email = email;
        Id = id;
    }

    // Overriding Equals(), otherwise Contains() won't work, GetHashCode() is also required
    public override bool Equals(object obj)
    {
        if (obj == null || GetType() != obj.GetType())
        {
            return false;
        }
        else
        {
            Employee employee = (Employee)obj;
            return (
                this.Name == employee.Name &&
                this.Surname == employee.Surname &&
                this.Position == employee.Position &&
                this.Email == employee.Email &&
                this.Id == employee.Id
            );
        }
    }

    public override int GetHashCode() { return Id.GetHashCode(); }

    public bool ValidateData() { return EmployeeValidator.ValidateAll(this.Name, this.Surname, this.Position, this.Email, this.Id); }

    public String Show() { return string.Format("Name: {0}, Surname: {1}, Position: {2}, Email: {3},  Id: {4}", this.Name, this.Surname, this.Position, this.Email, this.Id); }

    public bool IsMatch(Employee employee) { return employee.Equals(this); }

    public override string ToString() { return string.Format("Name: {0}, Surname: {1}, Position: {2}, Email: {3}, Id: {4}", this.Name, this.Surname, this.Position, this.Email, this.Id); }

}

partial class EmployeeValidator
{

    const int MIN_NAME_LENGTH = 2;
    const int MAX_NAME_LENGTH = 20;
    const int MIN_POSITION_LENGTH = 2;
    const int MAX_POSITION_LENGTH = 20;
    const int MIN_ID = 0;
    const String NAME_REGEX = @"^[a-zA-Z]+$";
    const String EMAIL_REGEX = "^[A-Za-z0-9.]+@(.+)$";  // Regex pattern : any number of letters, numbers and dots, then @, 
                                                        // then any number of letters, numbers and dots and $ asserting the end of the string

    // validate name and surname
    private static bool ValidateName(String name)
    {
        return !String.IsNullOrEmpty(name) && name.Length > MIN_NAME_LENGTH && name.Length < MAX_NAME_LENGTH && Regex.IsMatch(name, NAME_REGEX);
    }

    // validate email address against regex
    private static bool ValidateEmail(String email)
    {
        return Regex.IsMatch(email, EMAIL_REGEX);
    }

    private static bool ValidatePosition(String position)
    {
        return !String.IsNullOrEmpty(position) && position.Length > MIN_POSITION_LENGTH && position.Length < MAX_POSITION_LENGTH && Regex.IsMatch(position, NAME_REGEX);
    }

    private static bool ValidateId(int id)
    {
        return id > MIN_ID;
    }

    public static bool ValidateAll(String name, String surname, String position, String email, int id)
    {
        return (
            ValidateName(name) &&
            ValidateName(surname) &&
            ValidatePosition(position) &&
            ValidateEmail(email) &&
            ValidateId(id)
            );
    }
}

class Program
{

    bool Test() 
    {
        CatalogGeneric<Employee> catalog = new CatalogGeneric<Employee>();

        catalog.AddMany(new List<Employee>() {
            new Employee("Geralt", "Wiedzmin", "Wiedzmin", "geralt@gmail.com", 001),
            new Employee("Triss", "Merigold", "Czarodziejka", "triss@gmail.com", 002),
            new Employee("Sigismund", "Dijkstra", "Szpieg", "sigi@gmail.com", 003)
        });

        return catalog.Validate(new Employee("Triss", "Merigold", "Czarodziejka", "triss@gmail.com", 002));
    }

    static void Main(string[] args)
    {
        Program program = new();
        Console.WriteLine(program.Test());
    }

}


### Zadanie 3

Dodaj do kartoteki z pierwszego zadania odczytywanie i zapisywanie danych w formacie TXT, XML oraz JSON (minimum jeden). Wykorzystaj do tego celu wzorzec projektowy budowniczy. Abstrakcję konkretnego zapisu można przekazać do konstruktora kartoteki, która posiada metodę `Save()`.


In [5]:
using System.Text.Json;
using System.Text.RegularExpressions;
using System.Xml.Serialization;

class CatalogGeneric<T>
{
    private List<T> catalog = [];
    private readonly IReader<T> reader;
    private readonly IWriter<T> writer;

    public CatalogGeneric(IWriter<T> writer, IReader<T> reader)
    {
        this.writer = writer;
        this.reader = reader;
    }

    public void Add(T item) { catalog.Add(item); }
    public void AddMany(List<T> items) { catalog.AddRange(items); }
    public void Remove(T item) { catalog.Remove(item); }

    public void Print()
    {
        if (catalog.Count == 0)
        {
            Console.WriteLine("Catalog is empty");
        }
        else
        {
            foreach (T item in catalog)
            {
                Console.WriteLine(item?.ToString());
            }
        }
    }

    public T? Search(T catalogItem) { return (catalog.Count != 0 && catalog.Contains(catalogItem)) ? catalogItem : default(T); }
    public void Clear() { catalog.Clear(); }
    public bool Validate(T item) { return catalog.Contains(item); }
    public void Write() { writer.Write(catalog); }
    public void Read() { catalog = reader.Read(); }

}

public class Employee
{

    public string Name { get; set; }
    public string Surname { get; set; }
    public string Position { get; set; }
    public string Email { get; set; }
    public int Id { get; set; }

    public Employee()
    {
    }

    public Employee(string name, string surname, string position, string email, int id)
    {
        Name = name;
        Surname = surname;
        Position = position;
        Email = email;
        Id = id;
    }

    // Overriding Equals(), otherwise Contains() won't work, GetHashCode() is also required
    public override bool Equals(object obj)
    {
        if (obj == null || GetType() != obj.GetType())
        {
            return false;
        }
        else
        {
            Employee employee = (Employee)obj;
            return (
                this.Name == employee.Name &&
                this.Surname == employee.Surname &&
                this.Position == employee.Position &&
                this.Email == employee.Email &&
                this.Id == employee.Id
            );
        }
    }

    public override int GetHashCode() { return Id.GetHashCode(); }

    public bool ValidateData() { return EmployeeValidator.ValidateAll(this.Name, this.Surname, this.Position, this.Email, this.Id); }

    public String Show() { return string.Format("Name: {0}, Surname: {1}, Position: {2}, Email: {3},  Id: {4}", this.Name, this.Surname, this.Position, this.Email, this.Id); }

    public bool IsMatch(Employee employee) { return employee.Equals(this); }

    public override string ToString() { return string.Format("Name: {0}, Surname: {1}, Position: {2}, Email: {3}, Id: {4}", this.Name, this.Surname, this.Position, this.Email, this.Id); }

}
partial class EmployeeValidator
{

    const int MIN_NAME_LENGTH = 2;
    const int MAX_NAME_LENGTH = 20;
    const int MIN_POSITION_LENGTH = 2;
    const int MAX_POSITION_LENGTH = 20;
    const int MIN_ID = 0;
    const String NAME_REGEX = @"^[a-zA-Z]+$";
    const String EMAIL_REGEX = "^[A-Za-z0-9.]+@(.+)$";  // Regex pattern : any number of letters, numbers and dots, then @, 
                                                        // then any number of letters, numbers and dots and $ asserting the end of the string

    private static bool ValidateName(String name)
    {
        return !String.IsNullOrEmpty(name) && name.Length > MIN_NAME_LENGTH && name.Length < MAX_NAME_LENGTH && Regex.IsMatch(name, NAME_REGEX);
    }

    private static bool ValidateEmail(String email)
    {
        return Regex.IsMatch(email, EMAIL_REGEX);
    }

    private static bool ValidatePosition(String position)
    {
        return !String.IsNullOrEmpty(position) && position.Length > MIN_POSITION_LENGTH && position.Length < MAX_POSITION_LENGTH && Regex.IsMatch(position, NAME_REGEX);
    }

    private static bool ValidateId(int id) { return id > MIN_ID; }

    public static bool ValidateAll(String name, String surname, String position, String email, int id)
    {
        return (
            ValidateName(name) &&
            ValidateName(surname) &&
            ValidatePosition(position) &&
            ValidateEmail(email) &&
            ValidateId(id)
            );
    }
}



public interface IReader<T>
{
    List<T> Read();
}

public class JSONReader<T> : IReader<T>
{
    public List<T> Read()
    {
        if (File.Exists("employee_catalog.json"))
        {
            string jsonString = File.ReadAllText("employee_catalog.json");

            List<T>? deserializedList = JsonSerializer.Deserialize<List<T>>(jsonString);
            return deserializedList ?? [];
        }
        else
        {
            return [];
        }
    }
}

public class XMLReader<T> : IReader<T>
{
    public List<T> Read()
    {
        if (File.Exists("employee_catalog.xml"))
        {
            XmlSerializer serializer = new XmlSerializer(typeof(List<T>));
            using (FileStream fileStream = File.OpenRead("employee_catalog.xml"))
            {
                List<T>? deserializedList = serializer.Deserialize(fileStream) as List<T>;
                return deserializedList ?? [];
            }
        }
        else
        {
            return [];
        }
    }
}

public interface IWriter<T>
{
    void Write(List<T> list);
}

public class JSONWriter<T> : IWriter<T>
{
    public void Write(List<T> list)
    {
        Console.WriteLine("Writing to JSON");
        string jsonString = JsonSerializer.Serialize(list);

        //print jsonString for debugging
        Console.WriteLine(jsonString);

        File.WriteAllText("employee_catalog.json", jsonString);
    }
}

public class XMLWriter<T> : IWriter<T>
{
    public void Write(List<T> list)
    {
        XmlSerializer serializer = new XmlSerializer(typeof(List<T>));
        using (FileStream fileStream = File.Create("employee_catalog.xml"))
        {
            serializer.Serialize(fileStream, list);
        }
    }
}

class Program
{
    public bool TestJSON()
    {
        CatalogGeneric<Employee> catalog = new CatalogGeneric<Employee>(new XMLWriter<Employee>(), new XMLReader<Employee>());

        catalog.AddMany(new List<Employee>() {
            new Employee("Geralt", "Wiedzmin", "Wiedzmin", "geralt@gmail.com", 001),
            new Employee("Triss", "Merigold", "Czarodziejka", "triss@gmail.com", 002),
            new Employee("Sigismund", "Dijkstra", "Szpieg", "sigi@gmail.com", 003)
        });
        catalog.Write();
        catalog.Clear();
        catalog.Read();

        return catalog.Validate(new Employee("Triss", "Merigold", "Czarodziejka", "triss@gmail.com", 002));

    }

    public bool TestXML()
    {
        CatalogGeneric<Employee> catalog = new CatalogGeneric<Employee>(new XMLWriter<Employee>(), new XMLReader<Employee>());

        catalog.AddMany(new List<Employee>() {
            new Employee("Geralt", "Wiedzmin", "Wiedzmin", "geralt@gmail.com", 001),
            new Employee("Triss", "Merigold", "Czarodziejka", "triss@gmail.com", 002),
            new Employee("Sigismund", "Dijkstra", "Szpieg", "sigi@gmail.com", 003)
        });
        catalog.Write();
        catalog.Clear();
        catalog.Read();

        return catalog.Validate(new Employee("Triss", "Merigold", "Czarodziejka", "triss@gmail.com", 002));

    }

    static void Main(string[] args)
    {
        Program program = new Program();

        Console.WriteLine("JSON: " + program.TestJSON());
        Console.WriteLine("XML: " + program.TestXML());
    }

}


### Zadanie 4
 
Rozszerz poprzednie zadanie o możliwość szyfrowania danych metodą cezara. Blok przesunięć powinien być większy, niż jeden. W celu jego przechowywania wykorzystaj klasę `ConfigurationManager` (najpierw należy dodać referencję do biblioteki `System.Configuration`).

In [2]:
using System.Text.Json;
using System.Text.RegularExpressions;
using System.Xml.Serialization;
using System.Configuration;
using System.Text;
using System.Reflection.Metadata.Ecma335;

class CatalogGeneric<T>
{
    private List<T> catalog = [];
    private readonly IReader<T> reader;
    private readonly IWriter<T> writer;

    public CatalogGeneric(IWriter<T> writer, IReader<T> reader)
    {
        this.writer = writer;
        this.reader = reader;
    }

    public void Add(T item) { catalog.Add(item); }
    public void AddMany(List<T> items) { catalog.AddRange(items); }
    public void Remove(T item) { catalog.Remove(item); }

    public void Print()
    {
        if (catalog.Count == 0)
        {
            Console.WriteLine("Catalog is empty");
        }
        else
        {
            foreach (T item in catalog)
            {
                Console.WriteLine(item?.ToString());
            }
        }
    }

    public T? Search(T catalogItem) { return (catalog.Count != 0 && catalog.Contains(catalogItem)) ? catalogItem : default(T); }
    public void Clear() { catalog.Clear(); }
    public bool Validate(T item) { return catalog.Contains(item); }
    public void Write() { writer.Write(catalog); }
    public void Read() { catalog = reader.Read(); }

}

public class Employee
{

    public string Name { get; set; }
    public string Surname { get; set; }
    public string Position { get; set; }
    public string Email { get; set; }
    public int Id { get; set; }

    public Employee()
    {
    }

    public Employee(string name, string surname, string position, string email, int id)
    {
        Name = name;
        Surname = surname;
        Position = position;
        Email = email;
        Id = id;
    }

    // Overriding Equals(), otherwise Contains() won't work, GetHashCode() is also required
    public override bool Equals(object obj)
    {
        if (obj == null || GetType() != obj.GetType())
        {
            return false;
        }
        else
        {
            Employee employee = (Employee)obj;
            return (
                this.Name == employee.Name &&
                this.Surname == employee.Surname &&
                this.Position == employee.Position &&
                this.Email == employee.Email &&
                this.Id == employee.Id
            );
        }
    }

    public override int GetHashCode() { return Id.GetHashCode(); }

    public bool ValidateData() { return EmployeeValidator.ValidateAll(this.Name, this.Surname, this.Position, this.Email, this.Id); }

    public String Show() { return string.Format("Name: {0}, Surname: {1}, Position: {2}, Email: {3},  Id: {4}", this.Name, this.Surname, this.Position, this.Email, this.Id); }

    public bool IsMatch(Employee employee) { return employee.Equals(this); }

    public override string ToString() { return string.Format("Name: {0}, Surname: {1}, Position: {2}, Email: {3}, Id: {4}", this.Name, this.Surname, this.Position, this.Email, this.Id); }

}

partial class EmployeeValidator
{

    const int MIN_NAME_LENGTH = 2;
    const int MAX_NAME_LENGTH = 20;
    const int MIN_POSITION_LENGTH = 2;
    const int MAX_POSITION_LENGTH = 20;
    const int MIN_ID = 0;
    const String NAME_REGEX = @"^[a-zA-Z]+$";
    const String EMAIL_REGEX = "^[A-Za-z0-9.]+@(.+)$";  // Regex pattern : any number of letters, numbers and dots, then @, 
                                                        // then any number of letters, numbers and dots and $ asserting the end of the string


    private static bool ValidateName(String name)
    {
        return !string.IsNullOrEmpty(name) && name.Length > MIN_NAME_LENGTH && name.Length < MAX_NAME_LENGTH && Regex.IsMatch(name, NAME_REGEX);
    }

    private static bool ValidateEmail(String email) { return Regex.IsMatch(email, EMAIL_REGEX); }

    private static bool ValidatePosition(String position)
    {
        return !string.IsNullOrEmpty(position) && position.Length > MIN_POSITION_LENGTH && position.Length < MAX_POSITION_LENGTH && Regex.IsMatch(position, NAME_REGEX);
    }

    private static bool ValidateId(int id) { return id > MIN_ID; }

    public static bool ValidateAll(String name, String surname, String position, String email, int id)
    {
        return (
            ValidateName(name) &&
            ValidateName(surname) &&
            ValidatePosition(position) &&
            ValidateEmail(email) &&
            ValidateId(id)
            );
    }
}

public interface IReader<T> { List<T> Read(); }

public class JSONReader<T> : IReader<T>
{

    private CaesarCipher cipher = new();

    public List<T> Read()
    {
        if (File.Exists("employee_catalog.json"))
        {
            string encryptedJsonString = File.ReadAllText("employee_catalog.json");
            string decryptedJsonString = cipher.Decrypt(encryptedJsonString);
            List<T>? deserializedList = JsonSerializer.Deserialize<List<T>>(decryptedJsonString);

            return deserializedList ?? [];
        }
        else
        {
            return [];
        }

    }
}

public class XMLReader<T> : IReader<T>
{

    private readonly CaesarCipher cipher = new();

    public List<T> Read()
    {
        if (File.Exists("employee_catalog.xml"))
        {
            XmlSerializer serializer = new(typeof(List<T>));

            using FileStream fileStream = File.OpenRead("employee_catalog.xml");
            using StreamReader reader = new(fileStream);
            string encryptedXmlString = reader.ReadToEnd();
            string decryptedXmlString = cipher.Decrypt(encryptedXmlString);

            StringReader stringReader = new(decryptedXmlString);
            List<T>? deserializedList = (List<T>?)serializer.Deserialize(stringReader);

            return deserializedList ?? [];
        }
        else
        {
            return [];
        }
    }
}

public interface IWriter<T>
{
    void Write(List<T> list);
}

public class JSONWriter<T> : IWriter<T>
{

    private readonly CaesarCipher cipher = new();

    public void Write(List<T> list)
    {
        string jsonString = JsonSerializer.Serialize(list);
        string encryptedJsonString = cipher.Encrypt(jsonString);

        File.WriteAllText("employee_catalog.json", encryptedJsonString);
    }
}

public class XMLWriter<T> : IWriter<T>
{

    private readonly CaesarCipher cipher = new();

    public void Write(List<T> list)
    {
        XmlSerializer serializer = new(typeof(List<T>));
        using StringWriter stringWriter = new();
        serializer.Serialize(stringWriter, list);
        string encryptedXmlString = cipher.Encrypt(stringWriter.ToString());
        File.WriteAllText("employee_catalog.xml", encryptedXmlString);
    }
}


class CaesarCipher
{

    private int shift;

    public string Encrypt(string input)
    {
        this.shift = GenerateShift();

        StringBuilder encrypted = new();
        foreach (char c in input)
        {
            encrypted.Append((char)(c + shift));
        }
        return encrypted.ToString();
    }

    public string Decrypt(string input)
    {
        this.shift = GetShift();

        StringBuilder decrypted = new();
        foreach (char c in input)
        {
            decrypted.Append((char)(c - shift));
        }

        return decrypted.ToString();
    }

    public static int GenerateShift()
    {
        Random random = new();
        int shift = random.Next(2, 10);

        ConfigurationManager.AppSettings["shift"] = shift.ToString();

        return shift;

    }

    public static int GetShift()
    {
        return int.TryParse(ConfigurationManager.AppSettings["shift"], out int parsedShift)
            ? parsedShift
            : throw new ArgumentException("Invalid shift value in configuration");
    }

}

class Program
{

    public bool TestJSON()
    {
        CatalogGeneric<Employee> catalog = new CatalogGeneric<Employee>(new XMLWriter<Employee>(), new XMLReader<Employee>());

        catalog.AddMany(new List<Employee>() {
            new Employee("Geralt", "Wiedzmin", "Wiedzmin", "geralt@gmail.com", 001),
            new Employee("Triss", "Merigold", "Czarodziejka", "triss@gmail.com", 002),
            new Employee("Sigismund", "Dijkstra", "Szpieg", "sigi@gmail.com", 003)
        });
        catalog.Write();
        catalog.Clear();
        catalog.Read();

        return catalog.Validate(new Employee("Triss", "Merigold", "Czarodziejka", "triss@gmail.com", 002));

    }

    public bool TestXML()
    {
        CatalogGeneric<Employee> catalog = new CatalogGeneric<Employee>(new XMLWriter<Employee>(), new XMLReader<Employee>());

        catalog.AddMany(new List<Employee>() {
            new Employee("Geralt", "Wiedzmin", "Wiedzmin", "geralt@gmail.com", 001),
            new Employee("Triss", "Merigold", "Czarodziejka", "triss@gmail.com", 002),
            new Employee("Sigismund", "Dijkstra", "Szpieg", "sigi@gmail.com", 003)
        });
        catalog.Write();
        catalog.Clear();
        catalog.Read();

        return catalog.Validate(new Employee("Triss", "Merigold", "Czarodziejka", "triss@gmail.com", 002));

    }

    static void Main(string[] args)
    {
        Program program = new Program();

        Console.WriteLine("JSON: " + program.TestJSON());
        Console.WriteLine("XML: " + program.TestXML());
    }

}